In [ ]:
# default_exp df_group_aggregate

In [ ]:
# Gruppieren und aggregieren

In [ ]:
#hide
# Autoreload funktioniert nicht bei XDebug
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *

## Init Spark Context

In [ ]:
from udemy_spark.spark_core import *
spark = get_spark_session() # Session anlegen
spark # Ausgabe der wichtigsten Session Informationen

## Laden der Daten

In [ ]:
df_sales = spark.read.csv('data/sales_info.csv',inferSchema=True,header=True)
df_sales.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [ ]:
df_sales.show(3)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
+-------+-------+-----+
only showing top 3 rows



## Gruppieren und Aggregieren

In [ ]:
df_sales.groupBy("Company") # gibt ein GroupedData objekt zurück http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData
df_sales.groupBy("Company").mean().show() # weitere Methoden count, max, min, sum

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



Weitere Methoden unter:  http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module

In [ ]:
df_sales.groupBy("Company").agg({"*":"count"}).show()
df_sales.agg({"*":"count"}).show()

+-------+--------+
|Company|count(1)|
+-------+--------+
|   APPL|       4|
|   GOOG|       3|
|     FB|       2|
|   MSFT|       3|
+-------+--------+

+--------+
|count(1)|
+--------+
|      12|
+--------+



In [ ]:
df_sales.groupBy("Company").agg({"Sales":"max", "Sales":"mean","*":"count"}).show()

+-------+-----------------+--------+
|Company|       avg(Sales)|count(1)|
+-------+-----------------+--------+
|   APPL|            370.0|       4|
|   GOOG|            220.0|       3|
|     FB|            610.0|       2|
|   MSFT|322.3333333333333|       3|
+-------+-----------------+--------+



## Funktionen
Uebersicht http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#module-pyspark.sql.functions

In [ ]:
from pyspark.sql.functions import countDistinct, avg, stddev, format_number

In [ ]:
df_sales.select(countDistinct("Sales").alias("Distinct Sales")).show()

+--------------+
|Distinct Sales|
+--------------+
|            11|
+--------------+



In [ ]:
df_sales.select(avg("Sales")).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



### Genauigkeit formatieren

In [ ]:
df_select = df_sales.select(avg("Sales").alias("avg"))
df_select.select(format_number("avg",2)).show()
df_sales.select(format_number(avg("Sales"),2).alias("avg")).show() # oder gleich direkt in einer Zeile

+---------------------+
|format_number(avg, 2)|
+---------------------+
|               360.58|
+---------------------+

+------+
|   avg|
+------+
|360.58|
+------+



## Order BY
http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy

In [ ]:
df_sales.sort("Sales").show(2) # same as df_sales.orderBy("Sales").show(2)
df_sales.sort("Sales", ascending=False).show(2)
df_sales.sort(["Person","Sales"],ascending=[True,False]).show(2) #
df_sales.sort(df_sales.Sales).show(2)
df_sales.sort(df_sales.Sales.desc()).show(2)

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
+-------+-------+-----+
only showing top 2 rows

+-------+------+-----+
|Company|Person|Sales|
+-------+------+-----+
|     FB|  Carl|870.0|
|   APPL|  Mike|750.0|
+-------+------+-----+
only showing top 2 rows

+-------+------+-----+
|Company|Person|Sales|
+-------+------+-----+
|   APPL| Chris|350.0|
|   MSFT|   Amy|124.0|
+-------+------+-----+
only showing top 2 rows

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
+-------+-------+-----+
only showing top 2 rows

+-------+------+-----+
|Company|Person|Sales|
+-------+------+-----+
|     FB|  Carl|870.0|
|   APPL|  Mike|750.0|
+-------+------+-----+
only showing top 2 rows

